Run metadata (generated 2025-10-27 20:39:43Z)

- SCILLM_FORCE_HTTPX_STREAM=
- LITELLM_MAX_RETRIES=
- LITELLM_RETRY_AFTER=
- LITELLM_TIMEOUT=
- SCILLM_ALLOW_TOOLS_SMOKE=
- SCILLM_ALLOW_STREAM_SMOKE=

Run metadata (generated 2025-10-27 19:49:26Z)

- SCILLM_FORCE_HTTPX_STREAM=
- LITELLM_MAX_RETRIES=
- LITELLM_RETRY_AFTER=
- LITELLM_TIMEOUT=
- SCILLM_ALLOW_TOOLS_SMOKE=
- SCILLM_ALLOW_STREAM_SMOKE=

### Runtime setup
The following envs enable stable retries and quiet streaming.

- `SCILLM_FORCE_HTTPX_STREAM=1`
- `LITELLM_MAX_RETRIES=3`, `LITELLM_RETRY_AFTER=1`, `LITELLM_TIMEOUT=45`
- Requires `tenacity` installed for backoff.

In [1]:
import os
os.environ.setdefault('SCILLM_FORCE_HTTPX_STREAM','1')
os.environ.setdefault('LITELLM_MAX_RETRIES','3')
os.environ.setdefault('LITELLM_RETRY_AFTER','1')
os.environ.setdefault('LITELLM_TIMEOUT','45')
try:
    import tenacity  # noqa: F401
    print('tenacity: ok')
except Exception:
    print('tenacity missing — run: pip install tenacity')


tenacity: ok


# Advanced — Tools only (smoke)

### Why/When/Gotchas/Troubleshooting
- Why: call function tools (OpenAI-format) with SciLLM.
- When: structured actions, deterministic JSON I/O.
- Gotchas: rate limits (429) — set max_retries/retry_after/timeout; annotate tool schema clearly.
- Troubleshooting: set SCILLM_ALLOW_TOOLS_SMOKE=1 to run the cell; inspect printed tool_calls.

## Curl sanity (documentation)

        ```bash
        curl -X POST \
          $CHUTES_API_BASE/chat/completions \
          -H "Authorization: Bearer $CHUTES_API_KEY" \
          -H "Content-Type: application/json" \
          -d '{
            "model": "$CHUTES_VLM_MODEL",
            "messages": [{"role":"user","content":"Tell me a 250 word story."}],
            "stream": true,
            "max_tokens": 256,
            "temperature": 0.7
          }'
        ```

In [2]:
import os, json
from scillm import completion
tools=[{'type':'function','function':{'name':'ack','description':'Acknowledge','parameters':{'type':'object','properties':{'ok':{'type':'boolean'}},'required':['ok']}}}]
if os.getenv('SCILLM_ALLOW_TOOLS_SMOKE','0').lower() not in {'1','true','yes'}:
  print('Skipping tools smoke — set SCILLM_ALLOW_TOOLS_SMOKE=1 to run')
else:
  resp = completion(
  model=os.environ.get('CHUTES_TOOLS_MODEL', os.environ.get('CHUTES_MODEL_ADVANCED', os.environ['CHUTES_MODEL'])),
  api_base=os.environ['CHUTES_API_BASE'],
  api_key=None,
  custom_llm_provider='openai_like',
  extra_headers={'Authorization': f"Bearer {os.environ['CHUTES_API_KEY']}"},
  messages=[{'role':'user','content':'Call ack with ok=true'}],
  tools=tools,
  temperature=0,
  max_tokens=32,
  max_retries=3, retry_after=1, timeout=30,
)
  print(getattr(resp.choices[0], 'tool_calls', None))

Skipping tools smoke — set SCILLM_ALLOW_TOOLS_SMOKE=1 to run
